## Исследование надёжности заёмщиков — анализ банковских данных

**Краткое описание:**
В нашем распоряжении данные о платежостпособности клиентов банка с данными о самих клеинтах . Заказчиком является кредитный отдел банка. Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель проекта:** 
Определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

### Описание данных
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Первый взгляд на данные

In [1]:
import pandas as pd
clients = pd.read_csv('/datasets/data.csv')
clients.head(10) # просмотр данных в таблице

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
clients.info() #просмотор общей информации о массиве данных

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
clients.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


### Вывод

Таблица состоит из 12 столбцов и 21525 строк. Очевидно,что в двух столбцах имеются пропуски (2174 в каждом из двух: 'days_employed' и 'total_income'). Таблица содержит числовые и строковые данные. Также сразу заметны артефакты: отрицательные значения в столбце с трудовым стажем, что противоречит здравому смыслу, а также невообразимо большие значения для показателей трудового стажа в этом же столбце. Помимо прочего, в столбце dob_years имеются нулевые значения, происхождение которых мы постараемся выяснить в дальнейшем.

## Предобработка данных

### Обработка пропусков и испраление данных

In [4]:
print(clients.isnull().sum()) # подсчет пустых значений в датафрейме.

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


In [5]:
clients.describe() # просмотр общих сведений о числовых значениях в датафрейме

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [6]:
# проверка соответсвуют ли в точности пустые значения 'days_employed' пустым значениям 'total_income'
len(clients.loc[(clients['days_employed'].isnull()) & (clients['total_income'].isnull())])

2174

In [7]:
#Приводим впорядок столбец children: исправляем  отрицательные и аномально большие значения.
clients['children'] = clients['children'].abs()
clients.loc[clients['children'] == 20, 'children'] = 2
clients['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [8]:
# Приводим впорядок столбец days_employed: исправляем  отрицательные и аномально большие значения.
clients['days_employed'] = clients['days_employed'].abs()
days_employed_median = clients['days_employed'].median()
clients['days_employed'] = clients['days_employed'].fillna(value = days_employed_median)
clients.loc[clients['days_employed'] > 300000, 'days_employed'] = days_employed_median
clients

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,2194.220567,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,2194.220567,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


In [9]:
#заполним нулевые значения возраста, беря во внимание трудовой стаж, 
#прибавим к трудовому стажу примерный предполагаемый возраст начала трудовой деятельности, переведенный в дни
clients.loc[clients['dob_years'] == 0, 'dob_years'] = clients['days_employed']+23*365
clients['dob_years'] = clients['dob_years'].astype('int')
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [10]:
# избавимся от артефакта в столбце gender
clients = clients.loc[clients['gender'] != 'XNA']
clients['gender'].unique()

array(['F', 'M'], dtype=object)

In [11]:
clients['debt'].unique()

array([0, 1])

In [12]:
# заменим пустые значения в столбце total_income на значения медианы каждой категории занятости
clients = clients.copy()
median_table = clients.groupby(['income_type']).agg({'total_income' : 'median'})
 
for inc_type in median_table.index:
    clients.loc[clients['income_type'] == inc_type,'total_income'] =\
    clients.loc[clients['income_type'] == inc_type,'total_income'].\
    fillna(median_table.loc[inc_type, 'total_income'])

In [13]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
days_employed       21524 non-null float64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


### Вывод

Пропуски содержатся в двух столбцах: общий трудовой стаж и доход у одних и тех же потенциальных заемщиков. 

Очевидно, данные этих двух столбцов взаимосвязаны. Таким образом, пропуски в одном из столбцов влекут за собой пропуски в другом. 

Основываясь на том, что люди с пропусками в данных о трудовом стаже и доходе имеют определенный вид дохода в столбце 'income_type' (в том числе госслужащие, сотрудники, пенсионеры...), можно предположить, что пропуски в стобце 'days_employed' могут быть сязаны с рядом причин:
- данные утеряны, 
- заемщик не имет официального подтверждения трудового стажа и, как следствие, уровня дохода
- намеренно или ошибочно не предоставил данную информацию.

Пропуски заполнялись по следующему принципу:
- с учетом того, что данные трудового стажа не предполагают дальнейшего использования при выполнении посталвленных задач проекта, пропуски были заменены на значение медианы по столбцу. Значение в этом и других случаях было выбрано во избежание искажения среднего значения в большую сторону с учетом наличия в некоторых столбцах аномально больших значений.
- пропуски в данных по общему доходу заполнялись несколько иначе, т.к. данные в дальнейшем понадобятся для расчета зависимости между уровнем дохода и возвратом кредита в срок. Пропуски также заполнялись значением медианы, но расчитанной для конкретной категории занятости.
- еще один вида пропуска был обнаружен в данных с возрастом заемщиков в виде нулевых значений. В данном случае нулевые значения были заменены исходя из гипотезы, что человек получив высшее образование в 23 года начинает свою трудовую деятельность. Поэтому, проссумировав примерный возраст начала трудовой деятельности(23 года) и имеющийся трудовой стаж (days_employed) мы получили примерный возраст замещика.

### Замена типа данных

In [14]:
# переведем трудовой стаж из дней в годы для удобства восприятия
def days_to_years(row):
    try:
        return int(row/365)
    except:
        return row 

# переименуем столбец соответствующим образом
clients.rename(columns = {'days_employed':'years_employed'}, inplace=True)

# сначала округлим  значения days_employed в нужную сторону
clients = clients.round({'years_employed':0})

# для удобства восприятия переведем трудовой стаж в целые числа
clients.loc[clients['years_employed'], 'years_employed'] =\
clients.loc[clients['years_employed'], 'years_employed'].\
apply(days_to_years)

In [15]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21524 entries, 0 to 21524
Data columns (total 12 columns):
children            21524 non-null int64
years_employed      21524 non-null int64
dob_years           21524 non-null int64
education           21524 non-null object
education_id        21524 non-null int64
family_status       21524 non-null object
family_status_id    21524 non-null int64
gender              21524 non-null object
income_type         21524 non-null object
debt                21524 non-null int64
total_income        21524 non-null float64
purpose             21524 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.8+ MB


### Вывод

В датафреме отсутствует потребность замены вещественного типа данных на целочисленный. Единственным случаем, когда потребовалось заменить тип данных - это перевод трудового стажа в целые числа. С учетом того, что дробная часть не несет значительной ценности, данная мера была принята исключительно для удобства восприятия информации.

### Обработка дубликатов

In [16]:
#clients = clients.drop_duplicates().reset_index(drop=True)
clients.duplicated().sum()

31

In [17]:
#просмотр уникальных значений в стобцах показал наличие дублей в столбце 'education'
clients['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      667
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [18]:
#поиск дубликатов с учётом регистра
clients['education'] = clients['education'].str.lower()
clients = clients.drop_duplicates().reset_index(drop=True)
clients.duplicated().sum()

0

### Вывод

Поиск дубликатов состоял из двух частей:
- поиск и удаление грубых дубликатов
- приведение данных к нижнему регистру и удаление вновь образовавшихся дубликатов
В первом случае причин возникновения дубликтов может быть множество: начиная от подачи кредитной заявки несколько раз, заканчивая человеческим фактором и техническим сбоем системы.
Во втором случае причины возникновения дубликатов очевидна - из-за разности регистра в данных об образовании строки в данных изначально считались как различющиеся.

### Лемматизация

In [19]:
from pymystem3 import Mystem
m = Mystem()
clients['purpose_lemma'] = clients['purpose'].apply(m.lemmatize)

from collections import Counter
print(Counter(clients['purpose_lemma'].sum()))   

Counter({' ': 33604, '\n': 21479, 'недвижимость': 6355, 'покупка': 5899, 'жилье': 4465, 'автомобиль': 4310, 'образование': 4016, 'с': 2922, 'операция': 2607, 'свадьба': 2333, 'свой': 2233, 'на': 2223, 'строительство': 1878, 'высокий': 1375, 'получение': 1314, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 943, 'дополнительный': 907, 'заниматься': 905, 'сыграть': 770, 'проведение': 769, 'сдача': 651, 'семья': 639, 'собственный': 635, 'со': 628, 'ремонт': 610, 'подержанный': 488, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


In [20]:
# для достоверности определеним контекст употребления лемм в целях кредита
clients['purpose'].value_counts()

свадьба                                   794
сыграть свадьбу                           770
на проведение свадьбы                     769
операции с недвижимостью                  676
покупка коммерческой недвижимости         661
операции с жильем                         652
операции с коммерческой недвижимостью     651
покупка жилья для сдачи                   651
жилье                                     647
покупка жилья                             646
покупка жилья для семьи                   639
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           628
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              610
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [21]:
# исходя из увиденного, выделим 4 укрупненных категории целей получения кредита
def categorized(row):
    if 'свадьба' in row:
        return 'свадьба'
    if ('недвижимость' in row) or ('жилье' in row):
        return 'недвижимость'
    if 'автомобиль' in row:
        return 'автомобиль'
    return 'образование'

clients['category'] = clients['purpose_lemma'].apply(categorized)

### Вывод

Целью лемматизации была унификация получения целей кредита - приведение их к единым укрупненным категориям, что нам и удалось сделать. В процессе были выделены леммы, посчитана частота их употребления в столбце purpose, проверен контекст их употребления и в конечном итоге применена функция для присовонения новой унифицированной категории для каждой записи в новом столбце. Теперь мы можем перейти к поиску зависимости влияния целей кредита на его возврат в срок.

### Категоризация данных

In [22]:
# раздеим потенциальных замещиков на тех, у которых есть дети и нет
def children_status(row):
    if row == 0:
        return 'no'
    return 'yes'

clients['children_status'] = clients['children'].apply(children_status)

In [23]:
# категоризируем по уровеню дохода
# выделим три категории уровня дохода: низкий, средний, высокий
# для этого определим границы каждого из категорий
# для начала разделим всех заещиков на две части применив метод медианы
median = clients['total_income'].median()
# теперь применим функицю медианы для части замещиков, у которых доход ниже значения median
# примем это значение за вернхнюю границу заемщиков с низким доходом
margin_low_income = clients.loc[clients['total_income'] <= median, 'total_income'].median()
# применим аналогичный метод для определения верхней границы заемщиков со средним доходом
margin_middle_income = clients.loc[clients['total_income'] >= median, 'total_income'].median()

# напишем функцию, которая присвоит категорию доходов на основе расчитанных выше значений
def income_categorized(row):
    if row <= margin_low_income:
        return 'низкий'
    if row <= margin_middle_income:
        return 'средний'
    return 'высокий'

clients['income_type'] = clients['total_income'].apply(income_categorized)

In [24]:
def family_status_categorized(row):
    if ('вдовец / вдова' in row) or ('в разводе' in row) or ('Не женат / не замужем' in row):
        return 'не женат / не замужем'
    return row
    
clients['family_status_type'] = clients['family_status'].apply(family_status_categorized)
clients

,children,years_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,category,children_status,family_status_type
0,1,8438,42,высшее,0,женат / замужем,0,F,высокий,0,253875.639453,покупка жилья,"[покупка, , жилье, \n]",недвижимость,yes,женат / замужем
1,1,4025,36,среднее,1,женат / замужем,0,F,средний,0,112080.014102,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,yes,женат / замужем
2,0,5623,33,среднее,1,женат / замужем,0,M,средний,0,145885.952297,покупка жилья,"[покупка, , жилье, \n]",недвижимость,no,женат / замужем
3,3,4125,32,среднее,1,женат / замужем,0,M,высокий,0,267628.550329,дополнительное образование,"[дополнительный, , образование, \n]",образование,yes,женат / замужем
4,0,2194,53,среднее,1,гражданский брак,1,F,средний,0,158616.077870,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,no,гражданский брак
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21474,1,4529,43,среднее,1,гражданский брак,1,F,высокий,0,224791.862382,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,yes,гражданский брак
21475,0,2194,67,среднее,1,женат / замужем,0,F,средний,0,155999.806512,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",автомобиль,no,женат / замужем
21476,1,2113,38,среднее,1,гражданский брак,1,M,низкий,1,89672.561153,недвижимость,"[недвижимость, \n]",недвижимость,yes,гражданский брак
21477,3,3112,38,среднее,1,женат / замужем,0,M,высокий,1,244093.050500,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",автомобиль,yes,женат / замужем


### Вывод

Для поиска поледующий зависимостей нам понадобятся категоризированные данные:
- по наличию детей. Заемщики были распределены на тех у которых есть дети и нет детей
- семейное положение. Категории "женат / замужем" и "гражданский брак", т.к. каждая из них может иметь сови особенности. Так, например, гражданский брак и офциальный могут иметь различные юридические аспекты при оформлении кредита. Все остальные варианты были объеденены в категорию "не женат / не замужем"
- категория доходов. все доходы были поделены на три равные части и присвоены соответствующие категории

## Выявления зависимостей

С целью автоматизации определения зависимости возврата кредита в срок от каких-либо факторов создадим следующую функцию:

In [25]:
def return_debt(arg):
    '''
    функция принимает на вход параметр клиента arg,
    возвращает таблицу с полем процентам возврата кредита return_debt
    клиентами сагрегированными по параметру arg
    '''
    pivot = clients.pivot_table(index=arg, 
                                     columns='debt', 
                                     values='gender', 
                                     aggfunc='count')
    pivot['sum'] = pivot.sum(axis=1)
    pivot['return_debt'] = (pivot[0] / pivot['sum']) * 100
    display(pivot.sort_values(by = 'return_debt', ascending = False))

Теперь создадим список из интересующих нас параметров `argumets_to_check` и прогононим их через цикл для выявления зависимости каждого из них с фактом возврата кредита в срок 

In [26]:
argumets_to_check = ['children_status', 'family_status_type', 'income_type', 'category']

for i in argumets_to_check:
    return_debt(i)
    print()

debt,0,1,sum,return_debt
children_status,,,,
no,13050,1063,14113,92.467937
yes,6688,678,7366,90.795547


debt,0,1,sum,return_debt
family_status_type,,,,
женат / замужем,11424,931,12355,92.464589
не женат / не замужем,4543,422,4965,91.500504
гражданский брак,3771,388,4159,90.670834


debt,0,1,sum,return_debt
income_type,,,,
высокий,5416,432,5848,92.612859
низкий,5000,433,5433,92.030186
средний,9322,876,10198,91.410080


debt,0,1,sum,return_debt
category,,,,
недвижимость,10038,782,10820,92.772643
свадьба,2147,186,2333,92.027432
образование,3646,370,4016,90.786853
автомобиль,3907,403,4310,90.649652


### Выводы
#### Есть ли зависимость между наличием детей и возвратом кредита в срок?

По представленным данным можно сделать вывод об отсутствии твердой зависимости между наличием детей и возвратом кредита в срок. Так, с незначительным перевесом статистика по возврату кредита говорит в пользу заемщиков без детей - порядка 92,5% возврата кредита в срок. В то время как заемщики с детьми возращают кредит в срок в 90,8% случаев.
#### Есть ли зависимость между семейным положением и возвратом кредита в срок?
Гипотеза об особенностях гражданского и официального браков в некоторой степени подтвердилась. Как результат, мы можем наблюдать худший процент по возвращению кредита в срок именно у заемщиков из категории "гражданский брак". В то время как женатые  /замужние заемщики демонстрируют лучший показатель дисциплинированости по возврату кредита. ВОзможно это говорит о большем уровне отвественности.
#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок? 
Зависимостю возврата кредита в срок от уровня дохода минимальная. Прогнозируемо люди с высоким уровнем дохода лучше всего возвращают кредиты. Немногим хуже лисциплина у людей со средним доходом. Худший показатель у людей со средним доходом.
#### Как разные цели кредита влияют на его возврат в срок?
Наилучшим образом возвращаются кредиты взятые на недвижимость и свадьбу. Хуже всего - автомобильные кредиты и образование.

## Общий вывод по проекту

Исследовав 4 вида зависиомостей мы можем отметить незначительные различия в проценте возврата кредита и отсутствии явной зависимости. Тем не менее, различия есть. Наилучшим образом возвращаются кредиты, взятые на недвижимость, хуже всего - на образование и автомобили. Можно предположить, что на образование берут кредиты в основном студенты с отсутствием стабильного дохода.

Также, можно проследить некую зависимость относительного высокого процента возврата кредита, взятого на свадьбу. Возможно, это говорит о выском уровне ответственности и как следствие характеризуется высоким процентом возврата кредита в срок.  Это подтверждает и показатель возврата у людей со статусом 'женат/замужем'.

Инетересен тот факт, что люди со средним достатком хуже всех возвращают кредиты, в то время как у людей с высоким уровнем дохода прогнозируемо возникает меньше всего трудностей с выплатами по кредиту.

Помимо всего прочего можно отметить луший процент по возврату у заемщиков без детей. Возможно, это связано с тем что заемщики с детьми вынуждены распределять совокупный доход в пользу ребенка и как следствие какой-то процент заемщиков с детьми испытывает финансовые трудности.